# Code to demo 3D Cell Generation API

This code uses the [hra-3d-cell-generation-api](https://github.com/cns-iu/hra-3d-cell-generation-api) to get cell summaries for anatomical structures to be displayed in [ra-cell-type-populations-visualization](https://github.com/cns-iu/hra-cell-type-populations-visualization), which is in turn displayed on the [companion website](https://cns-iu.github.io/hra-cell-type-populations-supporting-information/). The code is also used for one of the figures.

## Import statements

In [4]:
import requests
import json
import seaborn as sns
import csv
import pandas as pd
from io import StringIO
from pprint import pprint

## Generate request params

In [5]:
# Choose organ and AS to get 3D cells for
# organ = "https://ccf-ontology.hubmapconsortium.org/objects/v1.2/VH_M_Kidney_L.glb"
organ = "https://cdn.humanatlas.io/digital-objects/ref-organ/heart-female/v1.3/assets/3d-vh-f-heart.glb"
as_name = "VH_F_left_ventricle"
max_cells = 6000

# Set request params
request_params = {
    "file": organ,
    "file_subpath": as_name,
    "num_nodes": max_cells,
    "node_distribution": {}
}

# {
#     "file": "https://cdn.humanatlas.io/digital-objects/ref-organ/kidney-female-left/v1.3/assets/3d-vh-f-kidney-l.glb",
#     "file_subpath": "VH_F_renal_pyramid_L_a",
#     "num_nodes": 10,
#     "node_distribution": {
#         "KEY1": 0.1,
#         "KEY2": 0.3
#     }
# }

### Get AS-CT counts from HRA LOD

In [6]:
# Get cell type distribution from HRA LOD
url = 'https://grlc.io/api-git/hubmapconsortium/ccf-grlc/subdir/hra-pop//cell_types_in_anatomical_structurescts_per_as?endpoint=https%3A%2F%2Flod.humanatlas.io%2Fsparql'
header = {"accept": "text/csv"}

# make the GET request and capture text
response = requests.get(url, headers=header)
csv_response = response.text

# convert to data frame
df_full = pd.read_csv(StringIO(csv_response))
pprint(df_full)

                organ                                             as  \
0     Large Intestine  http://purl.obolibrary.org/obo/UBERON_0001153   
1     Large Intestine  http://purl.obolibrary.org/obo/UBERON_0001153   
2     Large Intestine  http://purl.obolibrary.org/obo/UBERON_0001153   
3     Large Intestine  http://purl.obolibrary.org/obo/UBERON_0001153   
4     Large Intestine  http://purl.obolibrary.org/obo/UBERON_0001153   
...               ...                                            ...   
3997         Prostate  http://purl.obolibrary.org/obo/UBERON_8410027   
3998         Prostate  http://purl.obolibrary.org/obo/UBERON_8410027   
3999         Prostate  http://purl.obolibrary.org/obo/UBERON_8410027   
4000         Prostate  http://purl.obolibrary.org/obo/UBERON_8410027   
4001         Prostate  http://purl.obolibrary.org/obo/UBERON_8410027   

                      as_label     sex        tool            modality  \
0                       caecum  Female  celltypist  sc_transc

### Create params for hra-3d-cell-generation-api as JSON

In [7]:
# Subset to only get cells for the desired anatomical structure
as_label = 'heart left ventricle'
organ = "Heart"
sex = "Female"
tool = "azimuth"

subset_df = df_full[((df_full['as_label'] == as_label)) &
               (df_full['organ'] == organ) &
               (df_full['sex'] == sex) &
               (df_full['tool'] == tool)
               ]

# Extract cell labels and percentages for key-value pairs in JSON params
cell_information = subset_df.loc[:, ['cell_label', 'cell_percentage']]

# Create a dictionary where cell_label is the key and cell_percentage is the value
cell_label = cell_information['cell_label'].tolist()
cell_percentage = cell_information['cell_percentage'].tolist()
pairs = dict(zip(cell_label, cell_percentage))

### Optional: Only keep `top_n` cell types

In [8]:
pairs = dict(zip(cell_label, cell_percentage))

show_max: int = 7

# Step 1: Sort the dictionary by values in descending order and select the top 5
top_5_items = sorted(pairs.items(), key=lambda item: item[1], reverse=True)[
    :show_max]

# Step 2: Create a new dictionary from these items
top_n_dict = dict(top_5_items)

pairs = top_n_dict

### Add to JSON params

In [9]:
request_params['node_distribution'] = pairs

## Fetch 3D cell positions from API and export them to CSV

In [10]:
# set the URL
url = 'https://apps.humanatlas.io/api/v1/mesh-3d-cell-population'

# make the POST request 
request = requests.post(url, json=request_params)
response = request.text.split('\n')

pprint(f'request.text:\n {request.text}')

# export to CSV
with open('output/3d_cell_distribution.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    for line in response:
        writer.writerow(line.split(','))

('request.text:\n'
 ' x,y,z,Cell Type\n'
 '0.0700009,0.449886,-0.0235384,Ventricular Cardiomycoyte\n'
 '0.0649939,0.439342,-0.0933706,Ventricular Cardiomycoyte\n'
 '0.0867136,0.40398,-0.057209,Ventricular Cardiomycoyte\n'
 '0.0497335,0.414453,-0.0353651,Ventricular Cardiomycoyte\n'
 '0.0855186,0.425343,-0.0335798,Ventricular Cardiomycoyte\n'
 '0.045076,0.434561,-0.0988508,Ventricular Cardiomycoyte\n'
 '0.0996727,0.441697,-0.060772,Ventricular Cardiomycoyte\n'
 '0.0663229,0.464826,-0.0821581,Ventricular Cardiomycoyte\n'
 '0.0469931,0.470976,-0.0654887,Ventricular Cardiomycoyte\n'
 '0.0822124,0.414822,-0.0763803,Ventricular Cardiomycoyte\n'
 '0.0558452,0.442927,-0.0925541,Ventricular Cardiomycoyte\n'
 '0.0461602,0.469404,-0.0599126,Ventricular Cardiomycoyte\n'
 '0.0697778,0.466059,-0.0679561,Ventricular Cardiomycoyte\n'
 '0.0527855,0.468961,-0.0513879,Ventricular Cardiomycoyte\n'
 '0.068253,0.411238,-0.0353795,Ventricular Cardiomycoyte\n'
 '0.0492122,0.402231,-0.0846041,Ventricular Cardi

## Build bar graph

In [ ]:
df_bar = 